### Initial Classification Neural Network

Before using any other famous Deep Neural Networks such as Alexnet or Resnet. We experimented to build our own shallow neural network to see how minimum Convolutional Neural Network would perform classifying Ebru-Suminagashi Images. 

## Library Selections
In order to use state of the art toolset for this research paper we installed CUDA 10 , Pytorch 1.0, Python 3.7 and openCV 3.4 on ubuntu 16.04 with NVDIA 1080 GPU.  https://arxiv.org/pdf/1606.02228.pdf


In [1]:
# Import required libraries for this section
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import cv2

import torch
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [3]:
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of data set to use as test
validation_size = 0.5
test_validation_size = 0.4

transform = transforms.Compose([ transforms.Resize((256,256)), transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

data_set = dset.ImageFolder(root="data",transform=transform)
dataloader = torch.utils.data.DataLoader(data_set, batch_size=4,shuffle=True,num_workers=2)

# obtain training indices that will be used for test
num_data = len(data_set)
indices = list(range(num_data))
np.random.shuffle(indices)
split = int(np.floor(test_validation_size * num_data))
train_idx, test_idx = indices[split:], indices[:split]
num_train_data = len(test_idx)
split_validation = int(np.floor(validation_size * num_train_data))
test_idx, validation_idx = test_idx[split_validation:], test_idx[:split_validation]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
validation_sampler  = SubsetRandomSampler(validation_idx)
test_sampler  = SubsetRandomSampler(test_idx)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(data_set, batch_size=batch_size,
                                           sampler = train_sampler, num_workers=num_workers)
validation_loader = torch.utils.data.DataLoader(data_set, batch_size=batch_size, 
                                           sampler = test_sampler, num_workers=num_workers)
test_loader  = torch.utils.data.DataLoader(data_set, batch_size=batch_size, 
                                           sampler = test_sampler, num_workers=num_workers)

classes = ('ebrus','suminagashis')

Andrew Ng says previos era (the era before big data) traditional train/test/validation split was 60/20/20. If we still have small dataset traditional ratios still ok to use like our ebru dataset which we have 800 examples

In [4]:
print('Total number of data - ', num_data)
print('Training Data approximately %60 of the total - ', len(train_idx))
print('Validation Data approximately %20 of the total - ', len(validation_idx))
print('Test Data approximately %20 of the total - ', len(test_idx))

Total number of data -  799
Training Data approximately %60 of the total -  480
Validation Data approximately %20 of the total -  159
Test Data approximately %20 of the total -  160


In [9]:
images, labels = next(iter(train_loader))

In [10]:
images.shape

torch.Size([20, 3, 256, 256])

In [11]:
from torch.autograd import Variable
import torch.nn.functional as F

class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    #Our batch shape for input x is (3, 256, 256)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 128 * 128, 64)
        
        #64 input features, 2 output features for our 2 defined classes
        self.fc2 = torch.nn.Linear(64, 2)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 256, 256) to (18, 256, 256)
        x = F.relu(self.conv1(x))
        
        #Size changes from (18, 256, 256) to (18, 128, 128)
        x = self.pool(x)
        
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 128, 128) to (1, 294912)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 18 * 128 *128)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 294912) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

In [13]:
def outputSize(in_size, kernel_size, stride, padding):

    output = int((in_size - kernel_size + 2*(padding)) / stride) + 1

    return(output)

In [14]:
import torch.optim as optim

def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [26]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in validation_loader:
            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(validation_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [27]:
CNN = SimpleCNN()
trainNet(CNN, batch_size=32, n_epochs=5, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 32
epochs= 5
learning_rate= 0.001
Epoch 1, 12% 	 train_loss: 17.43 took: 17.90s
Epoch 1, 25% 	 train_loss: 3.89 took: 17.64s
Epoch 1, 37% 	 train_loss: 0.99 took: 17.86s
Epoch 1, 50% 	 train_loss: 0.73 took: 17.73s
Epoch 1, 62% 	 train_loss: 0.71 took: 17.94s
Epoch 1, 75% 	 train_loss: 0.51 took: 18.03s
Epoch 1, 87% 	 train_loss: 0.40 took: 17.67s
Epoch 1, 100% 	 train_loss: 0.31 took: 17.62s
Validation loss = 0.24
Epoch 2, 12% 	 train_loss: 0.30 took: 17.76s
Epoch 2, 25% 	 train_loss: 0.27 took: 17.52s
Epoch 2, 37% 	 train_loss: 0.38 took: 17.52s
Epoch 2, 50% 	 train_loss: 0.25 took: 17.55s
Epoch 2, 62% 	 train_loss: 0.25 took: 17.35s
Epoch 2, 75% 	 train_loss: 0.22 took: 17.32s
Epoch 2, 87% 	 train_loss: 0.22 took: 17.53s
Epoch 2, 100% 	 train_loss: 0.20 took: 17.71s
Validation loss = 0.13
Epoch 3, 12% 	 train_loss: 0.24 took: 17.59s
Epoch 3, 25% 	 train_loss: 0.23 took: 17.55s
Epoch 3, 37% 	 train_loss: 0.17 took: 17.43s
Epoch 3, 50% 	 train_l